In [8]:
from cryo_em_sbi import CryoEmSbi
import matplotlib.pyplot as plt

In [14]:
cryosbi = CryoEmSbi("config_square.json")

In [10]:
indices, images = cryosbi.simulate(1)

Running 10 simulations.: 100%|██████████| 10/10 [00:00<00:00, 1290.83it/s]


In [15]:
ind_pre, images_pre = cryosbi.preprocess(indices, images)

In [1]:
plt.imshow(images_pre[0].reshape(32, 32))

NameError: name 'plt' is not defined